In [81]:
import pandas as pd
import numpy as np
from make_df import create_df, make_inputs
from hardEM_0 import HEM_fit, HEM_predictions 




In [82]:
#covariates = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']

covariates = ['x1', 'x2', 'x3']

#dist = [[3.7, 0.7], [5.1, 2.1], [4.75, 0.9], [10.5, 2.1], [8.75, 3.9], [5.7, 0.9], [3.75, 0.5], [9.75, 3.11], [2.11, 0.9], [7.7, 2.9]]

dist = [[5, 2], [7, 3.5], [12.5, 1.8]]

cols = ['censoring_indicator', 'cure_label']

In [83]:
test_model_weights = [0.5, -3.5, -5.5, 3]

In [84]:
foo = create_df(covariates, dist, 1000, test_model_weights, 0.4)

In [85]:
censored_inputs = make_inputs(foo, 0, cols)

noncensored_inputs = make_inputs(foo, 1, cols)

In [86]:
len(censored_inputs)

495

In [87]:
fit = HEM_fit(censored_inputs, noncensored_inputs, 0.5, 1000, 'use_clustering')

In [88]:
weights = fit['model_weights']

In [89]:
weights

array([ 1.09841518, -0.07160092, -0.37385047,  0.0488128 ])

In [90]:
predictions = HEM_predictions(weights, foo, covariates)

In [91]:
from sklearn.metrics import accuracy_score
y_pred = predictions['pred']
y_true = foo['cure_label']
accuracy_score(y_true, y_pred)

accuracy_score(y_true, y_pred)

0.937

In [92]:
from sklearn.metrics import roc_auc_score
y_true = foo['cure_label']
y_scores = predictions['prob']
roc_auc_score(y_true, y_scores)

0.9800561726995641